Imports

In [227]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Dataset creation, scaling and manipulation

In [228]:
# Loading dataset
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
m, n_features = X.shape

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1234)

# Scale
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

# Numpy to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# Converting y's to column vector
y_train = y_train.reshape(y_train.shape[0], -1)
y_test = y_test.reshape(y_test.shape[0], -1)

Model

In [229]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size,1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegression(n_features)         

Loss and optimisation

In [230]:
loss_criterion = nn.BCELoss()
optimiser = torch.optim.SGD(model.parameters(),lr=0.2)

Training loop

In [231]:
n_iter=1000
for epoch in range(n_iter):
    y_hat = model(X_train)
    loss = loss_criterion(y_hat, y_train)
    
    loss.backward()
    
    optimiser.step()
    optimiser.zero_grad()
    
    if epoch%200 == 0:
        print(f'Epoch: {epoch}   Loss: {loss.item()} ')

Epoch: 0   Loss: 0.780479907989502 
Epoch: 200   Loss: 0.05485886335372925 
Epoch: 400   Loss: 0.0438900962471962 
Epoch: 600   Loss: 0.038718417286872864 
Epoch: 800   Loss: 0.035539355129003525 


Accuracy with Test set

In [232]:
with torch.no_grad():
    y_hat = model(X_test)
    y_hat_cls = y_hat.round()
    acc = y_hat_cls.eq(y_test).sum()/y_test.shape[0]
    print(f'Accuracy: {acc*100:.2f} %')

Accuracy: 96.49 %
